# 0) 필요한 모듈 설치

In [ ]:
! pip3 install --upgrade pandas

In [ ]:
! pip install geopy

# 1) 필요한 모듈 import

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams, style
from matplotlib import font_manager, rc
import os

pd.set_option('mode.chained_assignment', None) # Warning 방지용

# 2) 데이터 load

In [3]:
# 메가커 현황 데이터 로드

data = pd.read_csv('Megacoffee_utf8.csv', index_col=0, encoding='utf8', engine='python')
# index_col=0 : index 열 설정
# header=0 : 열 이름(헤더) 설정

# 'python'의 의미: 기본 엔진인 'c' 대신에 'python' 엔진을 사용하겠다는 의미입니다.
# 'python' 엔진은 좀 더 유연하지만 속도가 느릴 수 있습니다.
# 일반적으로 데이터에 특수 문자가 포함되어 있거나 복잡한 형식을 가진 경우 'python' 엔진을 사용하는 것이 유리합니다.


data.head() #작업내용 확인용 출력

,지역,주소,전화번호
매장명,,,
동서울대점,경기,"경기 성남시 수정구 복정로 73 (복정동), 1층",031-722-0111
동서울터미널점,서울,"서울 광진구 강변역로4길 10 (구의동, 강변역지너스타워)",02-454-1003
북서울꿈의숲점,서울,"서울 성북구 돌곶이로 220 (장위동, 꿈의숲코오롱하늘채아파트), 상가104동 107호",02-912-8982
삼육서울병원점,서울특별시,"서울특별시 동대문구 망우로 78, 1층(휘경동)",02-2213-2322
서서울호수공원점,서울,서울 양천구 남부순환로 432 (신월동),02-2690-6011


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2922 entries, 동서울대점 to 협재해수욕장점
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      2922 non-null   object
 1   주소      2922 non-null   object
 2   전화번호    2922 non-null   object
dtypes: object(3)
memory usage: 91.3+ KB


In [7]:
# 현재 행정구역으로 업데이트(2024.09.24 기준)
# 세종특별자치시 에는 '군구'없이 '행정동'만 가지고 있습니다.
# '강원도'의 정식 명칭은 '강원특별자치도'입니다.
# '전라북도'의 정식 명칭은 '전북특별자치도'입니다.

# 행정구역 딕셔너리
Korea_district = {
    '서울특별시': ['종로구', '중구', '용산구', '성동구', '광진구', 
                  '동대문구', '중랑구', '성북구', '강북구', '도봉구', 
                  '노원구', '은평구', '서대문구', '마포구', '양천구', 
                  '강서구', '구로구', '금천구', '영등포구', '동작구', 
                  '관악구', '서초구', '강남구', '송파구', '강동구'],
    
    '부산광역시': ['중구', '서구', '동구', '영도구', '부산진구', 
                  '동래구', '남구', '북구', '해운대구', '사하구', 
                  '금정구', '강서구', '연제구', '수영구', '사상구', '기장군'],

    '대구광역시': ['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군', '군위군'],

    '인천광역시': ['중구', '동구', '미추홀구', '연수구', '남동구', 
                  '부평구', '계양구', '서구', '강화군', '옹진군'],
    
    '광주광역시': ['동구', '서구', '남구', '북구', '광산구'],

    '대전광역시': ['동구', '중구', '서구', '유성구', '대덕구'],

    '울산광역시': ['중구', '남구', '동구', '북구', '울주군'],

    '세종특별자치시': ['세종시'],

    '경기도': ['수원시', '용인시', '고양시', '화성시', '성남시', 
              '부천시', '남양주시', '안산시', '평택시', '안양시', 
              '시흥시', '파주시', '김포시', '의정부시', '광주시',
              '하남시', '광명시', '군포시', '양주시', '오산시',
              '이천시', '안성시', '구리시', '의왕시', '포천시',
              '양평군', '여주시', '동두천시', '과천시',  '가평군', 
              '연천군'],

    '강원도': ['춘천시', '원주시', '강릉시', '동해시', '태백시', 
              '속초시', '삼척시', '홍천군', '횡성군', '영월군', 
              '평창군', '정선군', '철원군', '화천군', '양구군', 
              '인제군', '고성군', '양양군'],

    '충청북도': ['청주시', '충주시', '제천시', '보은군', '옥천군', 
               '영동군', '증평군', '진천군', '괴산군', '음성군', 
               '단양군'],

    '충청남도': ['천안시', '공주시', '보령시', '아산시', '서산시', 
               '논산시', '계룡시', '당진시', '금산군', '부여군', 
               '서천군', '청양군', '홍성군', '예산군', '태안군'],

    '전라북도': ['전주시', '군산시', '익산시', '정읍시', '남원시', 
               '김제시', '완주군', '진안군', '무주군', '장수군', 
               '임실군', '순창군', '고창군', '부안군'],

    '전라남도': ['목포시', '여수시', '순천시', '나주시', '광양시', 
               '담양군', '곡성군', '구례군', '고흥군', '보성군', 
               '화순군', '장흥군', '강진군', '해남군', '영암군', 
               '무안군', '함평군', '영광군', '장성군', '완도군', 
               '진도군', '신안군'],

    '경상북도': ['포항시', '경주시', '김천시', '안동시', '구미시', 
               '영주시', '영천시', '상주시', '문경시', '경산시', 
               '의성군', '청송군', '영양군', '영덕군', '청도군', 
               '고령군', '성주군', '칠곡군', '예천군', '봉화군', 
               '울진군', '울릉군'],

    '경상남도': ['창원시', '진주시', '통영시', '사천시', '김해시', 
               '밀양시', '거제시', '양산시', '의령군', '함안군', 
               '창녕군', '고성군', '남해군', '하동군', '산청군', 
               '함양군', '거창군', '합천군'],

    '제주특별자치도': ['제주시', '서귀포시']
}

# 3) 데이터 준비

## 3-1) 주소를 시도/군구 로 나누기

In [9]:
## 주소에서 시도, 군구 정보 분리

addr = pd.DataFrame(data['주소'].apply(lambda v: v.split()[:2]).tolist(),
                    columns=('시도', '군구'))

addr.head()  #작업내용 확인용 출력

,시도,군구
0,경기,성남시
1,서울,광진구
2,서울,성북구
3,서울특별시,동대문구
4,서울,양천구


In [11]:
addr['시도'].unique()

array(['경기', '서울', '서울특별시', '충청남도', '경기도', '인천', '인천광역시', '강원', '강원특별자치도',
       '강원도', '광주', '광주광역시', '대전', '대전광역시', '대구', '대구광역시', '경상북도',
       '부산광역시', '부산', '울산광역시', '울산', '세종특별자치시', '경남', '경상남도', '경북',
       '전라남도', '전남', '전라북도', '전북', '전북특별자치도', '충남', '충북', '충청북도',
       '제주특별자치도'], dtype=object)

In [13]:
#korea_district의 key값으로 리스트를 만든다.
district_keys = list(Korea_district.keys())

# addr['시도'].unique()의 값을 sido에 대입한다.
# district_keys에 들어있지 않은 경우 sido 값을 not_matcing에 입력한다.
non_matching = [sido for sido in addr['시도'].unique() if sido not in district_keys]

print(non_matching)

['경기', '서울', '인천', '강원', '강원특별자치도', '광주', '대전', '대구', '부산', '울산', '경남', '경북', '전남', '전북', '전북특별자치도', '충남', '충북']


In [15]:
## 표준 행정구역 이름으로 수정 :  경기 -> 경기도, 경남 -> 경상남도, ...
addr_aliases = {'경기':'경기도', '서울':'서울특별시', '인천':'인천광역시',
                '강원':'강원도', '강원특별자치도':'강원도', 
                '광주':'광주광역시', '대전':'대전광역시', '대구':'대구광역시', 
                '부산':'부산광역시', '울산':'울산광역시',
                '경남':'경상남도', '경북':'경상북도', '전남':'전라남도', '전북':'전라북도',
                '전북특별자치도':'전라북도', '충남':'충청남도', '충북':'충청북도'
                }

addr['시도']= addr['시도'].apply(lambda v: addr_aliases.get(v, v))

non_matching = [sido for sido in addr['시도'].unique() if sido not in district_keys]

print(non_matching)

[]


In [ ]:
addr['시도'].unique()

In [ ]:
addr['군구'].unique()

In [ ]:
# 딕셔너리의 value와 key를 반대로 매핑
district_mapping = {value: key for key, values in Korea_district.items() for value in values}
district_keys = list(Korea_district.keys())

non_matching2 = [sido2 for sido2 in addr['군구'].unique() if (sido2 not in district_mapping)]

print(non_matching2)

In [ ]:
# non_matching_gungu에 해당하고 '시도'가 '세종특별자치시'인 모든 행 찾기
result = addr[(addr['군구'].isin(non_matching2)) & (addr['시도'] == '세종특별자치시')]

result

In [ ]:
addr.loc[addr['군구'].isin(non_matching2) & (addr['시도'] == '세종특별자치시'), '군구'] = np.nan  # NaN으로 설정
# result.loc['군구'] = np.NaN
addr.iloc[2635:2656]

In [ ]:
# 딕셔너리의 value와 key를 반대로 매핑
district_mapping = {value: key for key, values in Korea_district.items() for value in values}
district_keys = list(Korea_district.keys())

non_matching2 = [sido2 for sido2 in addr['군구'].unique() if (sido2 not in district_mapping)]

print(non_matching2)

## 3-2 위도 경도 구하기

In [23]:
data['주소2'] = data['주소'].apply(lambda x: x.split(',')[0])
data.head()
data['주소3'] = data['주소'].apply(lambda x: x.split('(')[0])
data.head()

,지역,주소,전화번호,주소2,주소3
매장명,,,,,
동서울대점,경기,"경기 성남시 수정구 복정로 73 (복정동), 1층",031-722-0111,경기 성남시 수정구 복정로 73 (복정동),경기 성남시 수정구 복정로 73
동서울터미널점,서울,"서울 광진구 강변역로4길 10 (구의동, 강변역지너스타워)",02-454-1003,서울 광진구 강변역로4길 10 (구의동,서울 광진구 강변역로4길 10
북서울꿈의숲점,서울,"서울 성북구 돌곶이로 220 (장위동, 꿈의숲코오롱하늘채아파트), 상가104동 107호",02-912-8982,서울 성북구 돌곶이로 220 (장위동,서울 성북구 돌곶이로 220
삼육서울병원점,서울특별시,"서울특별시 동대문구 망우로 78, 1층(휘경동)",02-2213-2322,서울특별시 동대문구 망우로 78,"서울특별시 동대문구 망우로 78, 1층"
서서울호수공원점,서울,서울 양천구 남부순환로 432 (신월동),02-2690-6011,서울 양천구 남부순환로 432 (신월동),서울 양천구 남부순환로 432


In [19]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Nominatim 초기화
geolocator = Nominatim(user_agent="myGeocoder")

data_part = data.iloc[:50]

# 위경도를 얻기 위한 함수 정의
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            print(location.latitude, location.longitude)
            return location.latitude, location.longitude
        else:
            return None, None  # 주소가 유효하지 않을 경우
    except GeocoderTimedOut:
        time.sleep(1)
        return get_lat_long(address)  # 타임아웃 발생 시 재귀 호출

# '위도'와 '경도' 열 추가
data_part['위도'], data_part['경도'] = zip(*data_part['주소2'].apply(get_lat_long))

# 결과 출력
data_part.head()


37.619671 127.0461571
37.5897081 127.0620209
37.5291614 126.8326238
37.5448427 126.9515367
37.4882373 127.0120067
37.5152469 127.1040621
37.4826746 126.9530317
37.4824722 126.9412128
37.45811495 126.95216069831967
37.554538 126.9636814
37.5819304 126.9239316
37.6017392 126.954885
37.5522844 126.9087685
37.5538464 127.0437701
37.58396795 127.05363002560506
37.567572299999995 126.97986005893793
37.6240955 127.08963750593591
37.5396211 126.9733282
37.5551129 126.9657954
37.6397628 126.9169905
37.5297106 127.1149633
37.556834249999994 126.83738998476841
36.9119039 127.1394544
37.6492703 127.0138004
37.5658833 126.9859321
37.5193784 126.9314564
37.5693472 126.9686176
37.4964556 127.11348224039948
37.4956352 127.1187337
37.49632 127.1316
37.5244909 127.0235968
37.4783003 126.8825117
37.4771329 126.8849089
37.477854 126.880425
37.4802575 126.8789252
37.477854 126.880425


,지역,주소,전화번호,주소2,위도,경도
매장명,,,,,,
동서울대점,경기,"경기 성남시 수정구 복정로 73 (복정동), 1층",031-722-0111,경기 성남시 수정구 복정로 73 (복정동),NaN,NaN
동서울터미널점,서울,"서울 광진구 강변역로4길 10 (구의동, 강변역지너스타워)",02-454-1003,서울 광진구 강변역로4길 10 (구의동,NaN,NaN
북서울꿈의숲점,서울,"서울 성북구 돌곶이로 220 (장위동, 꿈의숲코오롱하늘채아파트), 상가104동 107호",02-912-8982,서울 성북구 돌곶이로 220 (장위동,37.619671,127.046157
삼육서울병원점,서울특별시,"서울특별시 동대문구 망우로 78, 1층(휘경동)",02-2213-2322,서울특별시 동대문구 망우로 78,37.589708,127.062021
서서울호수공원점,서울,서울 양천구 남부순환로 432 (신월동),02-2690-6011,서울 양천구 남부순환로 432 (신월동),37.529161,126.832624


In [25]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Nominatim 초기화
geolocator = Nominatim(user_agent="myGeocoder")

data_part = data.iloc[:50]

# 위경도를 얻기 위한 함수 정의
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            print(location.latitude, location.longitude)
            return location.latitude, location.longitude
        else:
            return None, None  # 주소가 유효하지 않을 경우
    except GeocoderTimedOut:
        time.sleep(1)
        return get_lat_long(address)  # 타임아웃 발생 시 재귀 호출

# '위도'와 '경도' 열 추가
data_part['위도'], data_part['경도'] = zip(*data_part['주소3'].apply(get_lat_long))

# 결과 출력
data_part.head()


37.5354326 127.0935432
37.6157379 127.0500699
37.5195455 126.8384331
37.6255545 126.918725
37.5448427 126.9515367
37.4882373 127.0120067
37.52641 127.1181368
37.4824722 126.9412128
37.45811495 126.95216069831967
37.5571314 126.9686222
37.5819304 126.9239316
37.6008507 126.9567729
37.5522844 126.9087685
37.5839449 127.0535993
37.5068934 126.8787724
37.650058 127.0260842
37.6240485 127.0895786
37.5557831 126.9677701
37.6437578 127.0302076
37.6397628 126.9169905
37.4868596 126.9477174
37.6759999 127.0873997
37.5960022 127.08320423677911
37.3722272 126.7271386
37.4853076 126.91645
37.5358019 126.8981214
37.5656885 126.98724979962262
37.5193784 126.9314564
37.5693472 126.9686176
37.5192331 127.0230573
37.48195 126.88525
37.4802575 126.8789252
37.4802575 126.8789252


,지역,주소,전화번호,주소2,주소3,위도,경도
매장명,,,,,,,
동서울대점,경기,"경기 성남시 수정구 복정로 73 (복정동), 1층",031-722-0111,경기 성남시 수정구 복정로 73 (복정동),경기 성남시 수정구 복정로 73,NaN,NaN
동서울터미널점,서울,"서울 광진구 강변역로4길 10 (구의동, 강변역지너스타워)",02-454-1003,서울 광진구 강변역로4길 10 (구의동,서울 광진구 강변역로4길 10,37.535433,127.093543
북서울꿈의숲점,서울,"서울 성북구 돌곶이로 220 (장위동, 꿈의숲코오롱하늘채아파트), 상가104동 107호",02-912-8982,서울 성북구 돌곶이로 220 (장위동,서울 성북구 돌곶이로 220,37.615738,127.050070
삼육서울병원점,서울특별시,"서울특별시 동대문구 망우로 78, 1층(휘경동)",02-2213-2322,서울특별시 동대문구 망우로 78,"서울특별시 동대문구 망우로 78, 1층",NaN,NaN
서서울호수공원점,서울,서울 양천구 남부순환로 432 (신월동),02-2690-6011,서울 양천구 남부순환로 432 (신월동),서울 양천구 남부순환로 432,37.519545,126.838433


In [27]:
data_part.head(50)

,지역,주소,전화번호,주소2,주소3,위도,경도
매장명,,,,,,,
동서울대점,경기,"경기 성남시 수정구 복정로 73 (복정동), 1층",031-722-0111,경기 성남시 수정구 복정로 73 (복정동),경기 성남시 수정구 복정로 73,NaN,NaN
동서울터미널점,서울,"서울 광진구 강변역로4길 10 (구의동, 강변역지너스타워)",02-454-1003,서울 광진구 강변역로4길 10 (구의동,서울 광진구 강변역로4길 10,37.535433,127.093543
북서울꿈의숲점,서울,"서울 성북구 돌곶이로 220 (장위동, 꿈의숲코오롱하늘채아파트), 상가104동 107호",02-912-8982,서울 성북구 돌곶이로 220 (장위동,서울 성북구 돌곶이로 220,37.615738,127.050070
삼육서울병원점,서울특별시,"서울특별시 동대문구 망우로 78, 1층(휘경동)",02-2213-2322,서울특별시 동대문구 망우로 78,"서울특별시 동대문구 망우로 78, 1층",NaN,NaN
서서울호수공원점,서울,서울 양천구 남부순환로 432 (신월동),02-2690-6011,서울 양천구 남부순환로 432 (신월동),서울 양천구 남부순환로 432,37.519545,126.838433
서울갈현초점,서울,"서울 은평구 갈현로 260 (갈현동), 1층",02-353-3321,서울 은평구 갈현로 260 (갈현동),서울 은평구 갈현로 260,37.625554,126.918725
서울공덕초교점,서울특별시,"서울특별시 마포구 만리재옛길 6, 1층(신공덕동)",02-712-5541,서울특별시 마포구 만리재옛길 6,"서울특별시 마포구 만리재옛길 6, 1층",37.544843,126.951537
서울교대사거리점,서울,"서울 서초구 서초중앙로 72 (서초동, 영빌딩)",02-588-7080,서울 서초구 서초중앙로 72 (서초동,서울 서초구 서초중앙로 72,37.488237,127.012007
서울교통회관점,서울,"서울 송파구 올림픽로 319 (신천동, 사단법인 서울특별시 교통회관)",02-418-1002,서울 송파구 올림픽로 319 (신천동,서울 송파구 올림픽로 319,37.526410,127.118137


In [ ]:
import pandas as pd
import folium


# 기본 맵 생성: 서울 시청 근처의 위도와 경도로 초기화
map_CB = folium.Map(location=[37.560284, 126.975334], zoom_start=15)

# 데이터프레임의 각 행을 순회
for i, store in data_part.iterrows():
    # 위도와 경도가 NaN이 아닌 경우에만 마커 추가
    if pd.notnull(store['위도']) and pd.notnull(store['경도']):
        folium.Marker(
            location=[store['위도'], store['경도']],  # 마커 위치 설정
            popup=i,  # 인덱스를 매장명으로 사용 (매장명이 인덱스에 있음)
            icon=folium.Icon(color='red', icon='star')  # 마커 아이콘 설정
        ).add_to(map_CB)  # 맵에 마커 추가

# 생성된 맵을 HTML 파일로 저장
map_CB.save('map_Mega.html')


In [ ]:
import webbrowser
webbrowser.open('C:/Users/user/0Data/0924_/map_Mega.html')

# SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape
# \대신 / 사용